In [ ]:
"""
GRASS GIS example
"""


obspoints = '/mnt/d/mystuff/pwgis/mod2demo/obspoints.shp'
mdtraster = '/mnt/d/mystuff/pwgis/mod2demo/mdt.tif'
outvis    = '/mnt/d/mystuff/pwgis/mod2demo/visibility.tif'

In [ ]:
import geopandas as gp
import os
from pycode.wenv import run_grass

In [ ]:
# Create GRASS GIS Location
workspace = os.path.dirname(outvis)
location  = 'loc_' + os.path.splitext(os.path.basename(outvis))[0]

gbase = run_grass(workspace, location=location, srs=mdtraster)

# Start GRASS GIS Session

import grass.script.setup as gsetup
gsetup.init(gbase, workspace, location, 'PERMANENT')

In [ ]:
from pycode.grstools import rst_to_grs
from pycode.grstools import grs_viewshed
from pycode.grstools import rstcalc
from pycode.grstools import grs_to_rst

In [ ]:
# Add DEM to GRASS GIS workspace

grsdem = rst_to_grs(mdtraster, 'dem')

# Read Observation Points Shape
points = gp.read_file(obspoints)

In [ ]:
points

In [ ]:
visibilities = []
for idx, row in points.iterrows():
    # Produce viewshed raster
    vrst = grs_viewshed(
        grsdem, (row.geometry.x, row.geometry.y),
        f"visrst_{str(idx)}",
        max_dist=10000
    )
    visibilities.append(vrst)

In [ ]:
# Sum all visibility rasters
fraster = rstcalc(" + ".join(visibilities), 'joinvis')

In [ ]:
# Export result
grs_to_rst(fraster, outvis)

In [ ]:
# Export all temp files

for r in visibilities:
    grs_to_rst(r, os.path.join(workspace, r + '.tif'))